In [1]:
# The model that you want to train from the Hugging Face hub
model_name = "mistralai/Mistral-7B-Instruct-v0.1"


import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

hub_name = "StarkWizard/Mistral-7b-instruct-cairo-instruct"



Load model from hub for inference

- If you just need inference, run this
- we load the model from HFace Hub in 4 bits


In [2]:
import torch
from transformers import AutoTokenizer, TextStreamer, GenerationConfig, BitsAndBytesConfig
from attention_sinks import AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.unk_token


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True,
)
model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path=hub_name,
                                             trust_remote_code=True,
                                             device_map={"": 0},
                                             attention_sink_size=4,
                                             quantization_config=bnb_config,
                                            attention_sink_window_size=252, # <- Low for the sake of faster generation
                                             )
model.eval()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

[Attention Sinks] Injected Position Shifting into 32 attention classes.
[Attention Sinks] Injected Attention Sink KV Cache into 1 model class.


MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )

---
Sampling Inference
---

In [3]:
from transformers import TextStreamer, GenerationConfig


#prompt = "Create an array and append some animal names"
#prompt = "give an exemple of constructor"
#prompt="create an array 'messages' that contains a u128, a u32, a u256"
#prompt = "create a structure for mailAccount"
#prompt = "create an array of felt and append 1 to the array"
#prompt = "create a felt and affect it a value of 1"

#prompt = "write a  contract that returns the fibonacci of the caller address"
#prompt = "write an empty contract template"
#prompt = "what are spans used for"
#prompt = "How do I know if an array is empty"
#prompt = "what makes Cairo special"
prompt = "Create an array and append some domestic animal string names"
prompt="write a simple contract that returns the fibonnacci of the caller's address, without recursivity"

text =f"""[INST]
<<SYS>>
provide only one solution and no other possible solution, stick to the main topic, do not introduce any new topics or new question not provided by the student.
Make sure the explanations never be longer than 100 words.Don’t justify your answers. <SYS>>

Question: I'm working in Cairo 1 :{prompt} 
[/INST]"""

text =f"[INST]You are a cairo 1 expert and know no other computer language,  {prompt} [/INST]"


input_ids = tokenizer.encode(text, return_tensors="pt").to(model.device)

with torch.no_grad():
    streamer = TextStreamer(tokenizer)
    generated_tokens = model.generate(
        input_ids,
        generation_config=GenerationConfig(
            # use_cache=True is required, the rest can be changed up.
            use_cache = True,
            min_new_tokens=1,
            max_new_tokens=1050,

            do_sample=True,
            top_p=0.9,
            
        
            repetition_penalty =1.1,
            temperature=0.00100,

            bos_token_id=model.config.bos_token_id,
            eos_token_id=model.config.eos_token_id,
            
            pad_token_id=tokenizer.unk_token_id,
        ),
        streamer=streamer,
    )
    # Decode the final generated text
    output_text = tokenizer.decode(generated_tokens[0], skip_special_tokens=True)

<s> [INST]You are a cairo 1 expert and know no other computer language,  write a simple contract that returns the fibonnacci of the caller's address, without recursivity 

[/INST]
```
#[starknet::contract]
mod fib {
   #[external(v0)]
   fn fib(self: ContractState) -> u256 {
       let mut prev = 0;
       let mut curr = 1;
       let mut sum = 0;
       loop {
           sum = curr + prev;
           if sum > 256 {
               return sum;
           }
           let sender = self.get_caller_address();
           prev = curr;
           curr = sum;
       }
   }
}
```
</s>


In [37]:
prompt="write a simple contract that returns the fibonnacci of the caller's address, without recursivity"

text =f"""[INST]
<<SYS>>
provide only one solution and no other possible solution, stick to the main topic, do not introduce any new topics or new question not provided by the student.
Make sure the explanations never be longer than 100 words.Don’t justify your answers. <SYS>>

Question: I'm working in Cairo 1 :{prompt} 
[/INST]"""

text =f"[INST]You are a cairo 1 expert and know no other computer language,  {prompt} [/INST]"

llm(text)


[INST]You are a cairo 1 expert and know no other computer language,  write a simple contract that returns the fibonnacci of the caller's address, without recursivity [/INST]
```
pragma solidity ^0.8.0;

contract Fibonacci {
   function fibonacci(uint256 address) public view returns (uint256) {
       uint256 a = 0;
       uint256 b = 1;
       uint256 c = 0;
       uint256 d = 1;
       uint256 sum;
       uint256 i;
       for (i = 0; i < 20; i++) {
           sum = a + b;
           a = b;
           b = c;
           c = d;
           d = sum;
       }
       return d;
   }
}
```
This contract uses a loop to calculate the fibonacci sequence up to the 20th number, and then returns the last number in the sequence, which is the fibonacci of the caller's address. The function is marked as `view` and `returns`, which means it doesn't modify the state of the contract.</s>


[{'generated_text': "[INST]You are a cairo 1 expert and know no other computer language,  write a simple contract that returns the fibonnacci of the caller's address, without recursivity [/INST]\n```\npragma solidity ^0.8.0;\n\ncontract Fibonacci {\n    function fibonacci(uint256 address) public view returns (uint256) {\n        uint256 a = 0;\n        uint256 b = 1;\n        uint256 c = 0;\n        uint256 d = 1;\n        uint256 sum;\n        uint256 i;\n        for (i = 0; i < 20; i++) {\n            sum = a + b;\n            a = b;\n            b = c;\n            c = d;\n            d = sum;\n        }\n        return d;\n    }\n}\n```\nThis contract uses a loop to calculate the fibonacci sequence up to the 20th number, and then returns the last number in the sequence, which is the fibonacci of the caller's address. The function is marked as `view` and `returns`, which means it doesn't modify the state of the contract."}]

---
Beam Inference

In [11]:

import transformers
import torch

model.eval()

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=200,

    num_beams=2,
    early_stopping=True,
     output_attentions=False,
      output_hidden_states=False,
      temperature=0.01

)
#prompt = 'In Cairo 1, write a full sample contract with a constructor that stores the caller into owner'
prompt ="In cairo 1, increment i 6 times using  loop, provide an explanation of the code"
prompt = "Create an array and append some domestic animal names and write a test to check the value of the item at index 2"
prompt = " what is an array span ?"
prompt = "give an exemple of a simplestorage contract with unit tests"
prompt = "give an exemple of a  contract stores in owner the value of the callers address, add one function that returns the fibonacci value of owner, explain the code"
prompt = "write an empty contract template"
text =f"""[INST]
<<SYS>>
Write explanations and Cairo 1 code  to solve the following coding problem that obeys the constraints and passes the example test cases. Explain the code. Only use Cairo syntax and no other language. Please wrap your code answer using ```
</SYS>>Question: I'm working in Cairo 1 :{prompt} 
[/INST]"""
with torch.no_grad():
    sequences = pipeline(
    text,

        num_return_sequences=1,
                bos_token_id=model.config.bos_token_id,
                eos_token_id=model.config.eos_token_id,
                pad_token_id=model.config.eos_token_id,

    )
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

/home/pechaut/miniconda3/envs/cairo-llm/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.01` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


KeyboardInterrupt: 

In [13]:
#prompt = "Create an array and append some animal names"
#prompt="create an array 'messages' that contains a u128, a u32, a u256"
#prompt = "create a structure for mailAccount"
#prompt = "create an array of felt and append 1 to the array"
#prompt = "create a felt and affect it a value of 1"
#prompt="create a function for fibonacci"
#prompt = "what are spans used for"
prompt = "How do I know if an array is empty"
#prompt = "what are spans used for"
prompt = "write a contract for implementing a function that computes u(n)=u(n-1)+1, with n passed as parameter"


text =f"[INST]I'm working in Cairo. You are a cairo expert and know no other computer language, answer in less than 170 words{prompt} [/INST]"

input_ids = tokenizer(text, return_tensors="pt", truncation=True).input_ids.cuda()

outputs = model.generate(input_ids=input_ids,       
    max_new_tokens=200,

    temperature=0.01,

    )


print(f"Prompt:\n{prompt}\n")
print(f"Answer:\n{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(text):]}")


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Prompt:
write a contract for implementing a function that computes u(n)=u(n-1)+1, with n passed as parameter

Answer:

```
#[starknet::contract]
mod u_contract {
    #[view]
    fn u(n: u64) -> u64 {
        n.saturating_add(1)
    }
}
```
This contract defines a single view function `u` that takes a single parameter `n` of type `u64` and returns the value of `u(n)` as a `u64`. The function uses the `saturating_add` method to ensure that the result is within the range of a `u64`.


In [4]:
from transformers import TextStreamer, GenerationConfig


prompt = "Create an array and append domestic animal names"
#prompt = "give an exemple of a simplestorage contract with unit tests"
#prompt = "create a felt and affect it a value of 1"
#prompt="In the example 'let y = { let x = 3; x + 1 };', what is the value of y? give a similar sample"
#prompt="How can you print a variable's value in Cairo? Give a full sample using an array"
#prompt = "hat type of operations can be considered expressions in Cairo? Give an exemple."
#prompt = "What is a characteristic feature of Felt252Dict<T> when interacting with it? Give a sample"
#prompt = "How do I create a span ? "
#prompt = "How do I know if an array is empty, give 2 examples"
#prompt = "what makes Cairo special"
#prompt = "Create an array and append some domestic animal names"
#prompt="write a contract that computes the fibonacci of caller's address and explain the weakness of the program"
#prompt="write a contract that returns the fibonacci of a value passed as parameter. implement the test function for the value 10"
#prompt="write an empty contract template and give the instructions to compile and deploy it"
#prompt="write a contract that stores the caller's address in its constructor and a function that returns the fibonnacci of that address"
#prompt="How can you implement the `Copy` trait on a custom type in Cairo?"
#prompt = "What's the significance of the `self` parameter in my method of the `CubeGeom` trait?"
#prompt = "write a contract with a function that returns the address of the caller"
#prompt="how can i cast a variable into another ?"

#prompt = "What do the terms 'parent' and 'child' mean in the context of modules?"
#prompt="What can be defined in trait and impl blocks related to a type?"
#prompt = "How do I know if an array is empty, give 2 examples"

#rompt="write a  basic simple contract that returns the fibonacci of the caller's address"
#prompt = "Create an array and append some domestic animal names as string"



#prompt = "How do I know if an array is empty, give 2 examples "

#prompt = "Provide a basic code snippet that defines a struct named 'User' with fields like 'active', 'username', and so on"


#prompt="write an empty contract template and give the instructions to compile and deploy it"

#prompt = "write a contract with a function that returns the address of the caller"
#prompt = "write a contract with a add_felt function that gets the sum of 2 felt parameters, add tests"
#prompt = "Create an array and append 3 domestic animal names as string and print the the second element"
#prompt = "write a contract that returns the fibonacci of the caller address"

prompt="write a  contract that provides a function that takes two arrays as parameter and store an array composed of the sum of the element of the arrays"
#prompt = "What's the difference between Nullable<T> and Option in terms of where the wrapped value is stored?"


#prompt = "How does the Cairo convention style name functions and variables?"

#prompt = "write a  contract that returns the fibonacci(caller address)"

#prompt = "What does the #[starknet::component] attribute signify?"

#prompt = " What is the naming convention for enum variants, give some examples"

#prompt ="""#[test]
#[available_gas(200000)]
#fn test_loop() {
#    let mut counter = 0;
#    //TODO make the test pass without changing any existing line
#    loop {
#        counter += 1;
#    };
#    assert(counter == 140, 'counter should be 140')
#}
#"""

#prompt = "write a contract for implementing a function that computes u(n)=u(n-1)+1, with n passed as parameter"
prompt="write an empty contract template and give the instructions to compile and deploy"

prompt = """
translate this Solidity contract into Cairo 1, and explain what you have done :
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.0;

/**
 * @title SimpleStorage
 * @dev Store & retrieve value in a variable
 */
contract SimpleStorage {
    uint256 private storedData;

    function store(uint256 num) public {
        storedData = num;
    }


    function retrieve() public view returns (uint256){
        return storedData;
    }
}

 """

text =f"""[INST]
<<SYS>>
A student asks you a question about Cairo 1. Provide a concise answer to the student's questions,do not expand the subject of the question, do not introduce any new topics or new question not provided by the student.
Make sure the explanations never be longer than 300 words.Don’t justify your answers. Don’t give information not mentioned in the CONTEXT INFORMATION.provide only one solution <SYS>>

Question: I'm working in Cairo 1 :{prompt} 
[/INST]"""



input_ids = tokenizer.encode(text, return_tensors="pt").to(model.device)

with torch.no_grad():
    streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
    generated_tokens = model.generate(
        input_ids,
        generation_config=GenerationConfig(
                       # use_cache=True is required, the rest can be changed up.
            use_cache=True,
            min_new_tokens=1,
            max_new_tokens=1050,

            top_k=4000,

            top_p=0.99,
            do_sample=True,


            temperature=0.4,

            bos_token_id=model.config.bos_token_id,
            eos_token_id=model.config.eos_token_id,
            pad_token_id=model.config.eos_token_id,
        ),
        streamer=streamer,
    )
    # Decode the final generated text
    output_text = tokenizer.decode(generated_tokens[0], skip_special_tokens=True)


```
// SPDX-License-Identifier: MIT

#[starknet::contract]
mod simple_storage {

   #[storage]
   struct Storage {
       stored_data: u256,
   }

   #[external(v0)]
   impl SimpleStorage of super::SimpleStorage {
       fn store(ref self: SimpleStorage, num: u256) {
           self.stored_data = num;
       }

       fn retrieve(self: @SimpleStorage) -> u256 {
           self.stored_data
       }
   }
}
```

